In [ ]:
import pandas as pd
import numpy as np


In [ ]:
from bs4 import BeautifulSoup as soup
from datetime import date, datetime
from urllib.request import Request, urlopen
 

In [ ]:
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import plotly.offline as py
import seaborn as sns
import gc
import warnings
warnings.filterwarnings("ignore")

from pandas_profiling import ProfileReport

*Web* Scrapping ofDATASET from worldometer.info


In [ ]:
today = datetime.now()
yesterday_data = "%s %d, %d" %(date.today().strftime('%b'), today.day-1, today.year)
yesterday_data

'Aug 24, 2021'

In [ ]:
url = "https://www.worldometers.info/coronavirus/#countries"
req = Request(url, headers={'User-Agent':"Mozilla/5.0"})

webpage = urlopen(req)
page = soup(webpage, 'html.parser')

#page

In [ ]:
table = page.findAll('table',{'id': 'main_table_countries_yesterday'})

containers = table[0].findAll('tr',{'style': ''})
title = containers[0]

del containers[0]

all_data = []
clean = True

for country in containers:
  country_data = []
  country_container = country.findAll('td')

  if country_container[1].text == 'china':
    continue
  for i in range(1, len(country_container)):
    final_feature = country_container[i].text
    if clean:
      if i!=1 and i!=len(country_container)-1:
        final_feature = final_feature.replace(",","")

        if final_feature.find('+') !=-1:
          final_feature = final_feature.replace("+", "")
          final_feature = float(final_feature)
        elif final_feature.find('-') !=-1:
          final_feature = final_feature_replace('-', '')
          final_feature = float(final_feature)*-1
      if final_feature =='N/A':
        final_feature =0
      elif final_feature =='' or final_feature== " ":
        final_feature = -1
      country_data.append(final_feature)
  
  all_data.append(country_data)


In [ ]:
all_data

[['World',
  '213955360',
  658684.0,
  '4463994',
  10464.0,
  '191438279',
  626156.0,
  '18053087',
  '112789',
  '27448',
  '572.7',
  -1,
  -1,
  -1,
  'All',
  '\n',
  -1,
  -1,
  -1,
  -1,
  -1],
 ['China',
  '94687',
  35.0,
  '4636 ',
  -1,
  '88417',
  96.0,
  '1634',
  '19',
  '66',
  '3',
  '160000000',
  '111163',
  '1439323776 ',
  'Asia',
  '15201',
  '310467',
  '9',
  '0.02',
  -1,
  '1'],
 ['USA',
  '38968925',
  147619.0,
  '648161 ',
  1134.0,
  '30618644',
  52047.0,
  '7702120',
  '24401',
  '116945',
  '1945',
  '570513424',
  '1712094',
  '333225477 ',
  'North America',
  '9',
  '514',
  '1',
  '443',
  '3',
  '23,114'],
 ['India',
  '32511370',
  51016.0,
  '435788 ',
  737.0,
  '31746626',
  55948.0,
  '328956',
  '8944',
  '23297',
  '312',
  '509391792',
  '365016',
  '1395531433 ',
  'Asia',
  '43',
  '3202',
  '3',
  '37',
  '0.5',
  '236'],
 ['Brazil',
  '20615008',
  31014.0,
  '575829 ',
  885.0,
  '19530843',
  50896.0,
  '508336',
  '8318',
  '96202'

In [ ]:
df = pd.DataFrame(all_data)
df.drop([15,16,17,18,19,20], inplace = True,axis =1)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,World,213955360,658684.0,4463994,10464.0,191438279,626156.0,18053087,112789,27448,572.7,-1,-1,-1,All
1,China,94687,35.0,4636,-1.0,88417,96.0,1634,19,66,3,160000000,111163,1439323776,Asia
2,USA,38968925,147619.0,648161,1134.0,30618644,52047.0,7702120,24401,116945,1945,570513424,1712094,333225477,North America
3,India,32511370,51016.0,435788,737.0,31746626,55948.0,328956,8944,23297,312,509391792,365016,1395531433,Asia
4,Brazil,20615008,31014.0,575829,885.0,19530843,50896.0,508336,8318,96202,2687,56580445,264038,214289417,South America


In [ ]:
column_labels = ['Country','Total cases', 'New cases', 'Total Deaths', 'New Deaths', 'Total Recovered', 'New recovered', 'Active cases','Serious cases', 'Total cases/1M', 'Total deaths/1M', 'Total tests','Tests/1M', 'Population', 'Continents'] 
df.columns = column_labels
df.head()

,Country,Total cases,New cases,Total Deaths,New Deaths,Total Recovered,New recovered,Active cases,Serious cases,Total cases/1M,Total deaths/1M,Total tests,Tests/1M,Population,Continents
0,World,213955360,658684.0,4463994,10464.0,191438279,626156.0,18053087,112789,27448,572.7,-1,-1,-1,All
1,China,94687,35.0,4636,-1.0,88417,96.0,1634,19,66,3,160000000,111163,1439323776,Asia
2,USA,38968925,147619.0,648161,1134.0,30618644,52047.0,7702120,24401,116945,1945,570513424,1712094,333225477,North America
3,India,32511370,51016.0,435788,737.0,31746626,55948.0,328956,8944,23297,312,509391792,365016,1395531433,Asia
4,Brazil,20615008,31014.0,575829,885.0,19530843,50896.0,508336,8318,96202,2687,56580445,264038,214289417,South America


In [ ]:
for label in df.columns:
  if label !='Country' and label != 'Continents':
    df[label] = pd.to_numeric(df[label])

In [ ]:
df['%inc new cases']=df['New cases']/df['Total cases']*100
df['%inc new deaths']=df['New Deaths']/df['Total Deaths']*100
df['%inc new recovered']=df['New recovered']/df['Total Recovered']*100


In [ ]:
df.head()

,Country,Total cases,New cases,Total Deaths,New Deaths,Total Recovered,New recovered,Active cases,Serious cases,Total cases/1M,Total deaths/1M,Total tests,Tests/1M,Population,Continents,%inc new cases,%inc new deaths,%inc new recovered
0,World,213955360,658684.0,4463994,10464.0,191438279,626156.0,18053087,112789,27448.0,572.7,-1,-1,-1,All,0.307860,0.234409,0.327080
1,China,94687,35.0,4636,-1.0,88417,96.0,1634,19,66.0,3.0,160000000,111163,1439323776,Asia,0.036964,-0.021570,0.108576
2,USA,38968925,147619.0,648161,1134.0,30618644,52047.0,7702120,24401,116945.0,1945.0,570513424,1712094,333225477,North America,0.378812,0.174957,0.169985
3,India,32511370,51016.0,435788,737.0,31746626,55948.0,328956,8944,23297.0,312.0,509391792,365016,1395531433,Asia,0.156917,0.169119,0.176233
4,Brazil,20615008,31014.0,575829,885.0,19530843,50896.0,508336,8318,96202.0,2687.0,56580445,264038,214289417,South America,0.150444,0.153691,0.260593


Expolatory Data Analysis

In [ ]:
cases = df[['Total Recovered','Active cases','Total Deaths']].loc[0]
cases_df = pd.DataFrame(cases).reset_index()
cases_df.columns = ["Type","Total"]

cases_df["Percentage"] = np.round(100*cases_df['Total']/np.sum(cases_df['Total']),2)
cases_df["Virus type"] = ["COVID-19" for i in range(len(cases_df))]

cases_df

,Type,Total,Percentage,Virus type
0,Total Recovered,191438279,89.48,COVID-19
1,Active cases,18053087,8.44,COVID-19
2,Total Deaths,4463994,2.09,COVID-19


In [ ]:
fig = px.bar(cases_df,x ='Virus type', y = 'Percentage', color = 'Type', hover_data = ["Total"], width = 600)
fig.show()

In [ ]:
cases = df[['New cases','New recovered','New Deaths']].loc[0]
cases_df = pd.DataFrame(cases).reset_index()
cases_df.columns = ["Type","Total"]

cases_df["Percentage"] = np.round(100*cases_df['Total']/np.sum(cases_df['Total']),2)
cases_df["Virus type"] = ["COVID-19" for i in range(len(cases_df))]

cases_df

,Type,Total,Percentage,Virus type
0,New cases,658684.0,50.85,COVID-19
1,New recovered,626156.0,48.34,COVID-19
2,New Deaths,10464.0,0.81,COVID-19


In [ ]:
fig = px.bar(cases_df,x ='Virus type', y = 'Percentage', color = 'Type', hover_data = ["Total"], width = 600)
fig.show()

In [ ]:
per = np.round(df[["%inc new cases","%inc new deaths","%inc new recovered"]].loc[0],2)

per_df = pd.DataFrame(per)
per_df.columns = ['Percentage']
#per_df["Virus type"] = ["COVID-19" for i in range(len(cases_df))]
per_df

,Percentage
%inc new cases,0.31
%inc new deaths,0.23
%inc new recovered,0.33


In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(x = per_df.index, y = per_df['Percentage'], marker_color = ["Yellow", 'blue','red']))
fig.show() 

Continents

In [ ]:
continents_df = df.groupby("Continents").sum().drop("All")  #groupby - continents and sum the numerics and drop rest(String type)
continents_df = continents_df.reset_index()  #indexing
continents_df.head()

,Continents,Total cases,New cases,Total Deaths,New Deaths,Total Recovered,New recovered,Active cases,Serious cases,Total cases/1M,Total deaths/1M,Total tests,Tests/1M,Population,%inc new cases,%inc new deaths,%inc new recovered
0,Africa,7655592,31644.0,191193,813.0,6742426,34109.0,721973,4753,850451.0,12762.8,64805619,6031968,1377213168,15.081558,-90.117518,7.831623
1,Asia,68310015,271335.0,1006538,4764.0,63610769,275350.0,3692707,41384,1849078.0,18256.0,1190795816,44343243,4648294430,32.676077,120.407200,30.428903
2,Australia/Oceania,151487,2233.0,1982,22.0,111316,3602.0,38188,464,196119.0,1724.0,33384813,2368690,41456390,5.744003,102.701069,9.969345
3,Europe,54449320,126305.0,1162846,1623.0,49458757,159587.0,3827717,10797,3982618.0,71039.0,1153888834,101846590,748146982,12.544763,-61.246302,9.792963
4,North America,46709324,181298.0,978253,1899.0,37063143,90623.0,8667926,31570,1814421.0,23715.0,643190368,39445945,594386449,32.489697,-222.026029,11.972785


In [ ]:
def continent_visulaization(v_list):
  for label in v_list:
    c_df = continents_df[['Continents',label]]
    c_df['Percentage'] = np.round(100*c_df[label]/np.sum(c_df[label]),2)
    c_df['Virus type'] = ["COVID-19" for i in range(len(c_df))]

    fig = px.bar(c_df,x ='Virus type', y = 'Percentage', color = 'Continents', hover_data = [label], width = 600)
    fig.update_layout(title = {"text" : label})
    fig.show()

In [ ]:
cases_list = ['Total cases', 'Active cases','New cases','Serious cases','Total cases/1M']

deaths_list = ['Total Deaths','New Deaths','Total deaths/1M']

recovered_list = ['Total Recovered', 'New recovered', '%inc new recovered']

In [ ]:
continent_visulaization(cases_list)
#continent_visulaization(deaths_list)
#continent_visulaization(recovered_list)

Countries

In [ ]:
#df = df.drop([len(df)-1])
country_df = df.drop([0])

country_df.head()

,Country,Total cases,New cases,Total Deaths,New Deaths,Total Recovered,New recovered,Active cases,Serious cases,Total cases/1M,Total deaths/1M,Total tests,Tests/1M,Population,Continents,%inc new cases,%inc new deaths,%inc new recovered
1,China,94687,35.0,4636,-1.0,88417,96.0,1634,19,66.0,3.0,160000000,111163,1439323776,Asia,0.036964,-0.021570,0.108576
2,USA,38968925,147619.0,648161,1134.0,30618644,52047.0,7702120,24401,116945.0,1945.0,570513424,1712094,333225477,North America,0.378812,0.174957,0.169985
3,India,32511370,51016.0,435788,737.0,31746626,55948.0,328956,8944,23297.0,312.0,509391792,365016,1395531433,Asia,0.156917,0.169119,0.176233
4,Brazil,20615008,31014.0,575829,885.0,19530843,50896.0,508336,8318,96202.0,2687.0,56580445,264038,214289417,South America,0.150444,0.153691,0.260593
5,Russia,6785374,18833.0,177614,794.0,6053503,18636.0,554257,2300,46473.0,1216.0,175700000,1203374,146006183,Europe,0.277553,0.447037,0.307855


In [ ]:
country = country_df.columns[1:14]
look_At = 5
fig = go.Figure() #creating fig obj
c = 0
for i in country_df.index :
  if c < look_At:
    fig.add_trace(go.Bar(name = country_df['Country'][i], x = country, y = country_df.loc[i][1:14]))
  else:
    break
  c+=1

fig.update_layout(title = {'text' : f'Top {look_At} countries affected'}, yaxis_type = 'log')   # we see if we only take numeric data, then high value dominates the other. therefore we will take log values.(normalize)
fig.show()